In [8]:
import numpy as np
import tensorflow as tf

import seaborn as sns
from matplotlib.pyplot import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import wfdb

import os

%matplotlib inline

sns.set(style='whitegrid', palette='muted')
rcParams['figure.figsize']=14,8
np.random.seed(1)
tf.random.set_seed(1)
print(tf.__version__)

import warnings
warnings.filterwarnings('ignore')

2.3.0


In [9]:
flist_normal = os.listdir('../data/qtData/Data/Normal/')
flist_abnormal = os.listdir('../data/qtData/Data/Abnormal')

flist_normal = [file for file in flist_normal if file.endswith('.dat')]
flist_abnormal = [file for file in flist_abnormal if file.endswith('.dat')]

In [10]:
records_normal={}
records_abnormal={}
for file in flist_normal:
    records_normal[file] = wfdb.rdrecord('../data/qtData/Data/Normal/'+file[:-4],channels=[1])
for file in flist_abnormal:
    records_abnormal[file] = wfdb.rdrecord('../data/qtData/Data/Abnormal/'+file[:-4],channels=[1])

In [13]:
series_norm={}
series_abnorm={}
for rec in records_normal:
    series_norm[rec]=np.ndarray.flatten(records_normal[rec].p_signal[:])
for rec in records_abnormal:
    series_abnorm[rec]=np.ndarray.flatten(records_abnormal[rec].p_signal[:])

In [19]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [21]:
shuffle_buffer_size=1000
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
train_set={}
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
for rec in records_normal:
    train_set[rec] = windowed_dataset(series_norm[rec], window_size=64, batch_size=100, shuffle_buffer=shuffle_buffer_size)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])


optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history={}
for rec in records_normal:
    history[rec] = model.fit(train_set[rec],epochs=500, callbacks=[callback])

Epoch 1/500
   2166/Unknown - 193s 89ms/step - loss: 0.0210 - mae: 0.1063

KeyboardInterrupt: 

In [22]:
import torch.

ModuleNotFoundError: No module named 'torch.util'